In [3]:
from selenium import webdriver
from random import randint

import sys, time
from random import randint
from bs4 import BeautifulSoup

chromedriver = "C:\selenium_driver_chrome\chromedriver.exe" #chromedriver.exe執行檔所存在的路徑
web = webdriver.Chrome(chromedriver)
web.get('https://www.dcard.tw/f/tmu?latest=true')
##web.set_window_position(0,0) #瀏覽器位置
web.set_page_load_timeout(20)
#get by class or get by id
#web.find_element_by_id("XXXXXXX").text

#get by css selector :
#check out the css selector method here... https://www.w3schools.com/cssref/css_selectors.asp
#web.find_element_by_css_selector("[name='btnK']").click()
tab = web.find_element_by_class_name("TabGroup_nav_1DEsl")
tab.click

i=0
while i < 15:
    time.sleep(randint(3,11))
    web.execute_script("window.scrollTo(0,50000);")
    i += 1
    #footer = web.find_element_by_class_name("PostList_bottom_3ENZt")
    #if footer.text == '已經沒有文章囉！':
        #break

#print(str(i))

content = web.page_source
c = BeautifulSoup(content, 'html.parser')
web.close()
tags = c.find_all('div', {'class':'PostEntry_container_245XM'})
print(tags)
lstArticle = []

for tag in tags:
    links = tag.find_all('a', href=True)
    for link in links:
        if link['href'] != None:
            h = link['href']
            lstArticle.append(''.join(h[:h.index('-')]))
            
web.quit()
print(len(lstArticle))

[<div class="PostEntry_container_245XM" data-reactid="495"><a class="PostEntry_entry_2rsgm" data-reactid="496" href="/f/tmu/p/226368114-問＃宿舍冷氣怎麼樣可以不問前一直吹"><div class="PostEntry_wrapper_2Zzy1" data-reactid="497"><div class="PostEntry_content_13O5x" data-reactid="498"><div class="PostEntry_contentLeft_20mhg" data-reactid="499"><strong class="PostEntry_title_t1BVP PostEntry_titleUnread_ycJL0" data-reactid="500">問＃宿舍冷氣怎麼樣可以不問前一直吹</strong><div class="PostEntry_excerpt_A0Bmh" data-reactid="501">請問各位大大有沒有人知道如何讓宿舍冷氣可以一直吹不用錢？之前聽學長姐說過什麼快速換卡有沒有人真的成功的，跪求神解！！！！！</div></div></div><footer class="PostEntry_footer_1ACZf" data-reactid="502"><div class="PostEntry_author_38yft" data-reactid="503"><span class="PostAuthor_root_3vAJf" data-reactid="504">臺北醫學大學</span></div><div class="PostEntry_comment_18ty5 PostEntry_commentUnread_1cVyd" data-reactid="505">8</div></footer><div class="PostEntry_badge_1lO_i" data-reactid="506"><div class="PostEntry_likeCount_19OXJ" data-reactid="507"><div class="PostLikeCount_

In [23]:
lstArticle = lstArticle[0:(len(lstArticle)-5)]
#lstArticle2

In [25]:
import pandas as pd
import urllib.request
import requests, sys, time
from bs4 import BeautifulSoup

url = "https://www.dcard.tw/f/tmu?latest=true"
domain = "https://www.dcard.tw"
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux i686; rv:47.0) Gecko/20100101 Firefox/47.0'}
#req = urllib.request.Request(url = url, headers=headers)
#page = urllib.request.urlopen(req)

lblM = ["PID","Poster","Gender","Title","PDate","PCont","PLike","PReply"]
dfMain = pd.DataFrame(columns = lblM)
lblD = ["PID","CIDX","Poster","Gender","CDate","Cmt","CLike"]
dfDetail = pd.DataFrame(columns = lblD)
#j for iterate test
#j = 0
Floor=0
PostID=""
try:
    for lst in lstArticle:
        url = domain + lst
        #print(url)
        req = urllib.request.Request(url = url, headers=headers)
        page = urllib.request.urlopen(req)
        content = page.read()
        bs = BeautifulSoup(content, 'html.parser')

        #Collect Info from poster
        lstMain = [] 
        #print(lst[lst.rindex('/')+1:])
        #"PID","Poster","Title","PDate","PCont","PLike","PReply"
        PostID = lst[lst.rindex('/')+1:]
        print(PostID)
        lstMain.append(''.join(PostID))
        lstMain.append(''.join(bs.find('div', {"class": "PostAuthorHeader_meta_331_g"}).div.span.text))
        if "female" in bs.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
            lstMain.append(''.join('f'))
        elif "male" in bs.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
            lstMain.append(''.join('m'))
        elif "nitial" in bs.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
            lstMain.append(''.join('i'))
        else:
            lstMain.append(''.join('o'))
        lstMain.append(''.join(bs.find('h1', {"class": "Post_title_1T3V5"}).text))
        lstMain.append(''.join(bs.find('div',{'class':'Post_published_13TGw'}).span.text))
        lstMain.append(''.join(bs.find('div',{'class':'Post_content_1xpMb'}).div.text))
        lstMain.append(''.join(bs.find('button',{'class':'PostFooter_likeBtn_jmo71'}).span.text[3:]))
        lstMain.append(''.join(bs.find('button',{'class':'PostFooter_commentBtn_X8ZXa'}).span.text[3:]))
        #Append list into dataframe    
        dfMain.loc[dfMain.shape[0]] = lstMain

        #Collect Info from comment
        bsCommentList = bs.find('div', {"class": "CommentList_container_3vk7F CommentList_padding_11kTy"})
        cList = bsCommentList.findAll('div',{'class':'CommentEntry_entry_3SaSr'})
        #"PID","CIDX","Poster","CDate","Cmt","CLike"
        for i, lvl in enumerate(cList, start=1):
            lstDetail = []
            lstDetail.append(''.join(lst[lst.rindex('/')+1:]))
            floor = i
            lstDetail.append(''.join(str(i)))
            lstDetail.append(''.join(lvl.find('div',{"class": "PostAuthorHeader_meta_331_g"}).div.span.text))
            if "female" in lvl.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
                lstDetail.append(''.join('f'))
            elif "male" in lvl.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
                lstDetail.append(''.join('m'))
            elif "nitial" in lvl.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
                lstDetail.append(''.join('i'))
            else:
                lstDetail.append(''.join('o'))
            lstDetail.append(''.join(lvl.find('span',{'class':'CommentEntry_date_2E4vY'}).span.text))
            lstDetail.append(''.join(lvl.find('div',{'class':'CommentEntry_content_1ATrw'}).text))
            if lvl.find('button',{'class':'CommentEntry_likeBtn_24ZQX'}) == None:
                lstDetail.append(''.join('0'))
            else:
                lstDetail.append(''.join(lvl.find('button',{'class':'CommentEntry_likeBtn_24ZQX'}).text))
            #Append list into dataframe
            dfDetail.loc[dfDetail.shape[0]] = lstDetail   

        #random sleep
        time.sleep(randint(3,10))

        #Mark the break if you are not in test mode
        #j+=1
        #if j%10 == 0:
            #print(PostID)
            #break
        #else:
            #continue
except Exception as e:
    print("The last PostID is :" + PostID + " Floor:" + str(Floor))
    print(e)
    
#print(dfMain) 
#print("------------------------------------") 
#print(dfDetail) 

226290629
226260030
226259854
226259324
226248081
226237450
226233084
226232309
226221030
226215427
226186885
226133162
226116100
226089564
226085179
226074765
226070607
226064857
226062650
226061564
226052071
226050193
226038768
226029680
226020865
226019629
225992834
225940676
225906407
225900712
225842206
225839970
225823176
225807626
225803924
225779600
225758192
225757879
225699530
225683015
225683014
225683013
225683012
225683011
225683010
225683009
225683008
225683007
225671175
225657232
225657231
225657230
225657229
225657228
225657227
225657226
225657225
225657224
225657223
225657222
225657221
225657220
225657219
225657218
225657217
225657216
225657215
225631844
225631843
225631842
225631841
225631840
225631839
225631838
225631837
225631836
225631835
225631834
225631833
225631832
225631831
225631830
225631829
225622167
225600301


In [27]:
dfMain.to_csv("MainPost_v_2.csv", sep=',')
dfDetail.to_csv("DetailContent_v_2.csv", sep=',')

In [23]:
from selenium import webdriver
import time
from random import randint

chromedriver = "C:\selenium_driver_chrome\chromedriver.exe" #chromedriver.exe執行檔所存在的路徑
web = webdriver.Chrome(chromedriver)

web.get('https://www.dcard.tw/f/tmu?latest=true')
web.set_window_position(0,0) #瀏覽器位置
web.set_page_load_timeout(20)
tab = web.find_element_by_class_name("TabGroup_nav_1DEsl")
tab.click

i=0
while i < 15:
    time.sleep(randint(4,11))
    web.execute_script("window.scrollTo(0,70000);")
    i += 1
    #footer = web.find_element_by_class_name("PostList_bottom_3ENZt")
    #if footer.text == '已經沒有文章囉！':
        #break

print(str(i))
web.close()

15


In [5]:
import csv
import os

#def ReadCSVasList(csv_file):
#    try:
#        with open(csv_file) as csvfile:
#            reader = csv.reader(csvfile, dialect='excel', quoting=csv.QUOTE_NONNUMERIC)
#            datalist = []
#            datalist = list(reader)
#            return datalist
#    except IOError as (errno, strerror):
#            print("I/O error({0}): {1}".format(errno, strerror))    
#    return

def WriteListToCSV(csv_file,csv_columns,data_list):
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.writer(csvfile, dialect='excel', delimiter=',', quoting=csv.QUOTE_NONE)
            #writer.writerow(csv_columns)
            for data in data_list:
                strTmp = str(data[:18])
                writer.writerow(strTmp)
    except (IOError, ValueError):
        print("An I/O error or a ValueError occurred")
    except:
        print("An unexpected error occurred")
        raise  
    return              

csv_columns = ['Link']

currentPath = os.getcwd()
#csv_file = "Articles.csv"
#print(csv_file)
#csv_data_list = ReadCSVasList(csv_file)
#print csv_data_list

# To Ignore 1st Row (Headers)          
#csv_data_list.pop(0)
#print csv_data_list

# append to list
#csv_data_list.append(['6', 'Suresh', 'India'])

#print csv_data_list


WriteListToCSV("Articles.csv",csv_columns,lstArticle)


In [17]:
dfMain.to_csv("MainPost_v_2.csv", sep=',')
dfDetail.to_csv("DetailContent_v_2.csv", sep=',')

In [24]:
lstArticle

['/f/tmu/p/226290629',
 '/f/tmu/p/226260030',
 '/f/tmu/p/226259854',
 '/f/tmu/p/226259324',
 '/f/tmu/p/226248081',
 '/f/tmu/p/226237450',
 '/f/tmu/p/226233084',
 '/f/tmu/p/226232309',
 '/f/tmu/p/226221030',
 '/f/tmu/p/226215427',
 '/f/tmu/p/226186885',
 '/f/tmu/p/226133162',
 '/f/tmu/p/226116100',
 '/f/tmu/p/226089564',
 '/f/tmu/p/226085179',
 '/f/tmu/p/226074765',
 '/f/tmu/p/226070607',
 '/f/tmu/p/226064857',
 '/f/tmu/p/226062650',
 '/f/tmu/p/226061564',
 '/f/tmu/p/226052071',
 '/f/tmu/p/226050193',
 '/f/tmu/p/226038768',
 '/f/tmu/p/226029680',
 '/f/tmu/p/226020865',
 '/f/tmu/p/226019629',
 '/f/tmu/p/225992834',
 '/f/tmu/p/225940676',
 '/f/tmu/p/225906407',
 '/f/tmu/p/225900712',
 '/f/tmu/p/225842206',
 '/f/tmu/p/225839970',
 '/f/tmu/p/225823176',
 '/f/tmu/p/225807626',
 '/f/tmu/p/225803924',
 '/f/tmu/p/225779600',
 '/f/tmu/p/225758192',
 '/f/tmu/p/225757879',
 '/f/tmu/p/225699530',
 '/f/tmu/p/225683015',
 '/f/tmu/p/225683014',
 '/f/tmu/p/225683013',
 '/f/tmu/p/225683012',
 '/f/tmu/p/

In [92]:
for lst in lstArticle:
    if lst[lst.rindex('/')+1:] in lstUsed:
        lstArticle.remove(lst)
    else:  
        print(lst)
        #break
    

/f/tmu/p/225376349
/f/tmu/p/225365371
/f/tmu/p/225361924
/f/tmu/p/225361697
/f/tmu/p/225361243
/f/tmu/p/225360824
/f/tmu/p/225355422
/f/tmu/p/225350087
/f/tmu/p/225349699
/f/tmu/p/225348231
/f/tmu/p/225340138
/f/tmu/p/225335591
/f/tmu/p/225332852
/f/tmu/p/225330264
/f/tmu/p/225311003
/f/tmu/p/225301883
/f/tmu/p/225300731
/f/tmu/p/225296055
/f/tmu/p/225281621
/f/tmu/p/225280599
/f/tmu/p/225276007
/f/tmu/p/225272907
/f/tmu/p/225265445
/f/tmu/p/225252118
/f/tmu/p/225251466
/f/tmu/p/225237460
/f/tmu/p/225230646
/f/tmu/p/225216419
/f/tmu/p/225214856
/f/tmu/p/225207938
/f/tmu/p/225203554
/f/tmu/p/225181851
/f/tmu/p/225158363
/f/tmu/p/225155822
/f/tmu/p/225153325
/f/tmu/p/225142138
/f/tmu/p/225138998
/f/tmu/p/225137955
/f/tmu/p/225126814
/f/tmu/p/225104001
/f/tmu/p/225099851
/f/tmu/p/225081793
/f/tmu/p/225077333
/f/tmu/p/225050912
/f/tmu/p/225038554
/f/tmu/p/225036929
/f/tmu/p/225030276
/f/tmu/p/225022223
/f/tmu/p/225014582
/f/tmu/p/225012678
/f/tmu/p/224980958
/f/tmu/p/224973943
/f/tmu/p/224

In [87]:
import csv

with open('Article.csv', 'r') as f:
  reader = csv.reader(f)
  lstArticle2 = list(reader)

print(lstArticle2)

[['Link'], ['/f/tmu/p/225736050'], ['/f/tmu/p/225699530'], ['/f/tmu/p/225683015'], ['/f/tmu/p/225683014'], ['/f/tmu/p/225683013'], ['/f/tmu/p/225683012'], ['/f/tmu/p/225683011'], ['/f/tmu/p/225683010'], ['/f/tmu/p/225683009'], ['/f/tmu/p/225683008'], ['/f/tmu/p/225683007'], ['/f/tmu/p/225671175'], ['/f/tmu/p/225657232'], ['/f/tmu/p/225657231'], ['/f/tmu/p/225657230'], ['/f/tmu/p/225657229'], ['/f/tmu/p/225657228'], ['/f/tmu/p/225657227'], ['/f/tmu/p/225657226'], ['/f/tmu/p/225657225'], ['/f/tmu/p/225657224'], ['/f/tmu/p/225657223'], ['/f/tmu/p/225657222'], ['/f/tmu/p/225657221'], ['/f/tmu/p/225657220'], ['/f/tmu/p/225657219'], ['/f/tmu/p/225657218'], ['/f/tmu/p/225657217'], ['/f/tmu/p/225657216'], ['/f/tmu/p/225657215'], ['/f/tmu/p/225631844'], ['/f/tmu/p/225631843'], ['/f/tmu/p/225631842'], ['/f/tmu/p/225631841'], ['/f/tmu/p/225631840'], ['/f/tmu/p/225631839'], ['/f/tmu/p/225631838'], ['/f/tmu/p/225631837'], ['/f/tmu/p/225631836'], ['/f/tmu/p/225631835'], ['/f/tmu/p/225631834'], ['/f/

In [89]:
#print(lstArticle2[0])
#lstArticle2.pop(0)
print(lstArticle2[0][0])


/f/tmu/p/225736050


In [90]:
lstArticle = []
for lst in lstArticle2:
    #print(lst[0])
    lstArticle.append(lst[0])

print(lstArticle)

['/f/tmu/p/225736050', '/f/tmu/p/225699530', '/f/tmu/p/225683015', '/f/tmu/p/225683014', '/f/tmu/p/225683013', '/f/tmu/p/225683012', '/f/tmu/p/225683011', '/f/tmu/p/225683010', '/f/tmu/p/225683009', '/f/tmu/p/225683008', '/f/tmu/p/225683007', '/f/tmu/p/225671175', '/f/tmu/p/225657232', '/f/tmu/p/225657231', '/f/tmu/p/225657230', '/f/tmu/p/225657229', '/f/tmu/p/225657228', '/f/tmu/p/225657227', '/f/tmu/p/225657226', '/f/tmu/p/225657225', '/f/tmu/p/225657224', '/f/tmu/p/225657223', '/f/tmu/p/225657222', '/f/tmu/p/225657221', '/f/tmu/p/225657220', '/f/tmu/p/225657219', '/f/tmu/p/225657218', '/f/tmu/p/225657217', '/f/tmu/p/225657216', '/f/tmu/p/225657215', '/f/tmu/p/225631844', '/f/tmu/p/225631843', '/f/tmu/p/225631842', '/f/tmu/p/225631841', '/f/tmu/p/225631840', '/f/tmu/p/225631839', '/f/tmu/p/225631838', '/f/tmu/p/225631837', '/f/tmu/p/225631836', '/f/tmu/p/225631835', '/f/tmu/p/225631834', '/f/tmu/p/225631833', '/f/tmu/p/225631832', '/f/tmu/p/225631831', '/f/tmu/p/225631830', '/f/tmu/p

In [ ]:
import requests, sys, urllib.request
from bs4 import BeautifulSoup

url = "https://www.dcard.tw/f/tmu?latest=true"
domain = "https://www.dcard.tw"
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux i686; rv:47.0) Gecko/20100101 Firefox/47.0'}
req = urllib.request.Request(url = url, headers=headers)
page = urllib.request.urlopen(req)
content = page.read()
c = BeautifulSoup(content, 'html.parser')
tags = c.find_all('div', {'class':'PostEntry_container_245XM'})
lstArticle = []

for tag in tags:
    links = tag.find_all('a', href=True)
    for link in links:
        if link['href'] != None:
            h = link['href']
            lstArticle.append(''.join(h[:h.index('-')]))

print(len(lstArticle))

In [14]:
import pandas as pd
import urllib.request
import requests, sys, time
from bs4 import BeautifulSoup

url = "https://www.dcard.tw/"
domain = "https://www.dcard.tw"
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux i686; rv:47.0) Gecko/20100101 Firefox/47.0'}
#req = urllib.request.Request(url = url, headers=headers)
#page = urllib.request.urlopen(req)

lblM = ["PID","Poster","Gender","Title","PDate","PCont","PLike","PReply"]
dfMain = pd.DataFrame(columns = lblM)
lblD = ["PID","CIDX","Poster","Gender","CDate","Cmt","CLike"]
dfDetail = pd.DataFrame(columns = lblD)
#j for iterate test
j = 0
Floor=0
PostID=""
lst = "/f/tmu/p/1215441"
url = domain + lst
#print(url)
req = urllib.request.Request(url = url, headers=headers)
page = urllib.request.urlopen(req)
content = page.read()
bs = BeautifulSoup(content, 'html.parser')

#Collect Info from poster
lstMain = [] 
#print(lst[lst.rindex('/')+1:])
#"PID","Poster","Title","PDate","PCont","PLike","PReply"
PostID = lst[lst.rindex('/')+1:]
print(PostID)
lstMain.append(''.join(PostID))
lstMain.append(''.join(bs.find('div', {"class": "PostAuthorHeader_meta_331_g"}).div.span.text))
if "female" in bs.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
    lstMain.append(''.join('f'))
elif "male" in bs.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
    lstMain.append(''.join('m'))
elif "nitial" in bs.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
    lstMain.append(''.join('i'))
else:
    lstMain.append(''.join('o'))
lstMain.append(''.join(bs.find('h1', {"class": "Post_title_1T3V5"}).text))
lstMain.append(''.join(bs.find('div',{'class':'Post_published_13TGw'}).span.text))
lstMain.append(''.join(bs.find('div',{'class':'Post_content_1xpMb'}).div.text))
lstMain.append(''.join(bs.find('button',{'class':'PostFooter_likeBtn_jmo71'}).span.text[3:]))
lstMain.append(''.join(bs.find('button',{'class':'PostFooter_commentBtn_X8ZXa'}).span.text[3:]))
#Append list into dataframe    
dfMain.loc[dfMain.shape[0]] = lstMain

#Collect Info from comment
bsCommentList = bs.find('div', {"class": "CommentList_container_3vk7F CommentList_padding_11kTy"})
cList = bsCommentList.findAll('div',{'class':'CommentEntry_entry_3SaSr'})
#"PID","CIDX","Poster","CDate","Cmt","CLike"
for i, lvl in enumerate(cList, start=1):
    lstDetail = []
    lstDetail.append(''.join(lst[lst.rindex('/')+1:]))
    floor = i
    lstDetail.append(''.join(str(i)))
    lstDetail.append(''.join(lvl.find('div',{"class": "PostAuthorHeader_meta_331_g"}).div.span.text))
    if "female" in lvl.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
        lstDetail.append(''.join('f'))
    elif "male" in lvl.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
        lstDetail.append(''.join('m'))
    elif "nitial" in lvl.find('div', {"class": "PostAuthorHeader_avatar_1V21V"}).next_element.attrs['class'][0]:
        lstDetail.append(''.join('i'))
    else:
        lstDetail.append(''.join('o'))
    lstDetail.append(''.join(lvl.find('span',{'class':'CommentEntry_date_2E4vY'}).span.text))
    lstDetail.append(''.join(lvl.find('div',{'class':'CommentEntry_content_1ATrw'}).text))
    if lvl.find('button',{'class':'CommentEntry_likeBtn_24ZQX'}) == None:
        lstDetail.append(''.join('0'))
    else:
        lstDetail.append(''.join(lvl.find('button',{'class':'CommentEntry_likeBtn_24ZQX'}).text))
    #Append list into dataframe
    dfDetail.loc[dfDetail.shape[0]] = lstDetail   

#random sleep
#time.sleep(randint(3,10))

#Mark the break if you are not in test mode
#j+=1
#if j%10 == 0:
    #print(PostID)
    #break
#else:
    #continue

    
#print(dfMain) 
#print("------------------------------------") 
#print(dfDetail) 

1215441


In [15]:
print(dfMain)
print(dfDetail)

       PID  Poster Gender         Title       PDate  \
0  1215441  臺北醫學大學      f  開學除了上課......  3月5日 12:22   

                                               PCont PLike PReply  
0  唉！開學兩週除了上課還有什麼活動是值得參加的嗎？我不想要繼續廢下去了！大家新學期都有什麼樣子...     8     14  
        PID CIDX              Poster Gender        CDate  \
0   1215441    1          臺北醫學大學 醫學系      m   3月5日 14:24   
1   1215441    2              臺北醫學大學      m   3月6日 07:31   
2   1215441    3     臺北醫學大學 高齡健康管理學系      f   3月6日 13:07   
3   1215441    4  臺北醫學大學 醫學檢驗暨生物技術學系      m   3月6日 15:06   
4   1215441    5         臺北醫學大學 牙醫學系      m   3月6日 17:41   
5   1215441    6       臺北醫學大學 醫務管理學系      m   3月6日 20:08   
6   1215441    7              臺北醫學大學      m   3月7日 05:27   
7   1215441    8              臺北醫學大學      f   3月7日 06:22   
8   1215441    9              臺北醫學大學      m   3月7日 12:58   
9   1215441   10              臺北醫學大學      m   3月8日 02:26   
10  1215441   11              臺北醫學大學      f   3月9日 04:22   
11  1215441   12          臺北醫學大學 